In [8]:
import pandas as pd
import numpy as np
import datetime
import joblib
from keras.models import load_model
from haversine import haversine
from urllib.parse import quote
import streamlit as st
from streamlit_folium import st_folium
import folium
import branca
from geopy.geocoders import Nominatim
import ssl
from urllib.request import urlopen
import plotly.express as px

# *********************** ▼ 가이드 코딩 Start ▼ ***********************

data = pd.read_csv('119_emergency_dispatch.csv', encoding="cp949")

data

,ID,출동일시,이름,성별,나이,체온,수축기 혈압,이완기 혈압,호흡 곤란,간헐성 경련,...,통증,만지면 아프다,무감각,마비,현기증,졸도,말이 어눌해졌다,시력이 흐려짐,중증질환,이송 시간
0,P10715,2023-09-30,성**,여자,65,36.9,102,66,1,0,...,0,0,0,0,0,1,0,0,심근경색,8
1,P10959,2023-10-26,표**,남자,86,36.7,106,72,0,0,...,0,1,0,0,0,0,0,0,복부손상,40
2,P11863,2023-10-05,변**,여자,21,38.1,120,77,0,0,...,0,1,0,0,0,0,0,0,복부손상,27
3,P12186,2023-10-17,도**,남자,57,36.5,129,75,0,0,...,0,0,0,0,0,1,0,0,뇌경색,26
4,P15043,2023-10-01,송**,여자,88,39.1,103,65,0,0,...,0,1,0,0,0,0,0,0,복부손상,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17205,P20482,2023-10-20,제갈**,여자,4,36.3,101,73,0,0,...,0,0,1,0,0,0,0,0,뇌경색,44
17206,P21742,2023-10-20,기**,여자,22,36.9,106,83,0,0,...,0,0,0,1,0,0,0,1,뇌출혈,33
17207,P22512,2023-10-20,표**,여자,65,36.9,95,78,0,0,...,0,0,1,0,0,0,1,0,뇌출혈,18
17208,P24482,2023-10-20,채**,남자,14,36.3,105,72,0,0,...,0,0,1,0,0,0,1,0,뇌출혈,60


In [13]:
hospital_list = pd.read_csv('daegu_hospital_list.csv')
hospital_list.head()
# hospital_list.columns

,hpid,dutyName,dutyAddr,dutyTel3,wgs84Lat,wgs84Lon,hperyn,hpopyn
0,A1300081,(재)미리내천주성삼성직수도회천주성삼병원,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",053-790-0555,35.840225,128.705733,10,4
1,A1300018,강남병원,"대구광역시 동구 동촌로 207, 강남병원 (방촌동)",053-980-9019,35.883009,128.663456,10,4
2,A1300002,경북대학교병원,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",053-200-5100,35.866235,128.604315,22,16
3,A1300003,계명대학교동산병원,대구광역시 달서구 달구벌대로 1035 (신당동),053-258-6301,35.853887,128.480130,28,24
4,A1300008,곽병원,대구광역시 중구 국채보상로 531 (수동),053-605-3333,35.870693,128.588728,10,4
